## Unit 06. 降維 (Dimensionality Reduction) 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 0. 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P2_Unit06_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_2' / 'Unit06'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# UMAP套件 (需要安裝: pip install umap-learn)
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("⚠️ UMAP套件未安裝，請執行: pip install umap-learn")

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定matplotlib中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 所有套件載入完成")

---
### **I. 練習題: 建立降維模型分析高維製程數據**
 - 使用下面化工製程高維監控數據集。
 - 數據包含20個製程變數，模擬複雜的化工製程監控場景。

In [ ]:
# 設定隨機種子
np.random.seed(42)

# 生成高維製程數據 (20個特徵)
n_samples = 1500

# 定義3種不同操作模式的中心點
centers = [
    [200, 10, 80, 350, 5, 95, 120, 2.5, 60, 150, 8, 75, 300, 4, 90, 110, 2.0, 55, 140, 7],  # 模式A: 高產量模式
    [180, 8, 70, 320, 4, 85, 100, 2.0, 50, 130, 6, 65, 280, 3, 80, 95, 1.5, 45, 120, 5],    # 模式B: 節能模式
    [220, 12, 90, 380, 6, 105, 140, 3.0, 70, 170, 10, 85, 320, 5, 100, 130, 2.5, 65, 160, 9] # 模式C: 高品質模式
]

# 生成3種操作模式的數據
data_list = []
labels_list = []
for i, center in enumerate(centers):
    n_mode = n_samples // 3
    mode_data = np.random.randn(n_mode, 20) * 5 + center  # 添加噪音
    data_list.append(mode_data)
    labels_list.extend([f'Mode_{chr(65+i)}'] * n_mode)  # Mode_A, Mode_B, Mode_C

# 合併所有數據
X = np.vstack(data_list)

# 創建DataFrame
feature_names = [
    'Reactor_Temperature', 'Reactor_Pressure', 'Feed_Flow_Rate', 
    'Cooling_Water_Temp', 'Catalyst_Concentration', 'Product_Purity',
    'Stirring_Speed', 'pH_Value', 'Residence_Time', 'Heating_Power',
    'Pressure_Drop', 'Conversion_Rate', 'Separator_Temp', 'Recycle_Flow',
    'Steam_Pressure', 'Condenser_Temp', 'Vapor_Flow', 'Liquid_Level',
    'Column_Pressure', 'Reflux_Ratio'
]

df = pd.DataFrame(X, columns=feature_names)
df['Operating_Mode'] = labels_list

# 添加一些額外的隨機樣本（模擬過渡狀態和異常點）
n_extra = 150
extra_data = np.random.randn(n_extra, 20) * 8 + np.mean(centers, axis=0)
extra_df = pd.DataFrame(extra_data, columns=feature_names)
extra_df['Operating_Mode'] = 'Transition'

# 合併數據
df = pd.concat([df, extra_df], ignore_index=True)

# 打亂數據順序
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 保存數據
df.to_csv(DATA_DIR/'process_monitoring_data.csv', index=False, encoding='utf-8-sig')
print(f"✓ 數據集已生成: {DATA_DIR}/process_monitoring_data.csv")
print(f"✓ 數據形狀: {df.shape}")
print(f"\n前5筆數據:\n{df.head()}")
print(f"\n操作模式分布:\n{df['Operating_Mode'].value_counts()}")

### **1. 數據準備與探索**
- 載入數據並檢查基本統計資訊
- 檢查數據維度和操作模式分布
- 繪製相關性熱圖（至少選取10個變數）
- 分析高維數據的挑戰
- **使用StandardScaler對特徵進行標準化**

### **2. PCA 降維分析**
- 使用PCA將數據降至2維
- 繪製解釋變異數比例圖（Explained Variance Ratio）
- 繪製累積解釋變異數圖
- 計算前2個主成分能解釋多少變異數
- 繪製2D散點圖（用顏色區分操作模式）
- 分析主成分的物理意義（查看loadings）

### **3. Kernel PCA 降維分析**
- 使用Kernel PCA（RBF核）將數據降至2維
- 嘗試不同的gamma參數（如0.1, 1, 10）
- 繪製2D散點圖（用顏色區分操作模式）
- 與線性PCA結果比較
- 分析哪種情況適合使用Kernel PCA

### **4. t-SNE 降維分析**
- 使用t-SNE將數據降至2維
- 嘗試不同的perplexity參數（如5, 30, 50）
- 設定合適的learning_rate和n_iter
- 繪製2D散點圖（用顏色區分操作模式）
- 觀察不同操作模式的分離程度
- **注意**: t-SNE不適合用於預測新數據

### **5. UMAP 降維分析**
- 使用UMAP將數據降至2維
- 嘗試不同的n_neighbors參數（如5, 15, 50）
- 嘗試不同的min_dist參數（如0.0, 0.1, 0.5）
- 繪製2D散點圖（用顏色區分操作模式）
- 比較UMAP與t-SNE的結果差異
- 分析UMAP在速度和結果上的優勢

### **6. 綜合比較與評估**
- 創建一個2x2的子圖，展示所有4種降維方法的結果
- 計算每種方法的Silhouette Score（針對操作模式標籤）
- 計算每種方法的計算時間
- 製作比較表格，包含：
  - 方法名稱
  - Silhouette Score
  - 計算時間
  - 優點
  - 缺點
  - 適用場景
- 分析哪種方法最適合這個化工製程監控數據

### **7. 降維後的應用：異常檢測**
- 使用PCA降維後的2D數據
- 計算每個樣本到其操作模式中心的距離
- 設定異常閾值（如3倍標準差）
- 標記異常點並在2D圖上以不同顏色顯示
- 分析哪些樣本被識別為異常
- 解釋降維如何幫助異常檢測

---
## II. 額外加分作業 (自由繳交)
- 學生可自由選擇是否繳交加分作業. (下週上課前完成email電子檔)
- 分數會加在最後總成績上, 每份作業加0.1 ~ 1.0分.
- 分數加的不多, 真的不一定要交加分作業, 正常出席上課做好期末報告即可.
- 加分作業不一定要全部都做完, 想繳交至少完成其中一項實驗即可.
- 請務必自行完成! 你可以問AI, 問同學, 問學長姊, 甚至參考以前別人的作業, 但一定要自行[吸收][執行][整理]結果!
- 不要貼AI的答案寄給老師看, 你自己看就好.
- 如果系統自動比對發現作業內容(與他人雷同, 原創性低, 抄襲比例過高), 作業的分數會0分.
- 如果你直接100%複製別人的作業繳交, 你會直接被當掉(提供作業給別人複製的也一樣).
- 老師看你作業要花時間, 真的有心想寫加分作業, 請好好自己做.

---
### 使用課堂作業所提供的化工製程監控數據集
- 嘗試修改生成模擬數據的程式
- 改變參數, 改變公式, 添加特徵, 增加操作模式 ... 等等.
- 產生獨一無二屬於你的數據集
- 使用降維方法進行分析
- 然後開始進行下面實驗

### **實驗1: PCA主成分數量選擇**

比較不同數量的主成分對後續分類效果的影響：
- **2個主成分**: 降至2D
- **3個主成分**: 降至3D
- **5個主成分**: 保留更多信息
- **10個主成分**: 高維保留

**要求**:
- 記錄每種情況的解釋變異數
- 使用KMeans對降維後數據進行分群
- 計算分群的Silhouette Score
- 繪製解釋變異數 vs 分群品質的關係圖
- **分析**: 多少個主成分最合適？為什麼？

### **實驗2: Kernel PCA 核函數比較**

比較不同核函數的效果：
- **Linear**: 線性核（等同於標準PCA）
- **RBF**: 徑向基函數核
- **Polynomial**: 多項式核（degree=2, 3）
- **Sigmoid**: Sigmoid核

**要求**:
- 對每種核函數降維至2D
- 繪製2D散點圖比較
- 計算分群的Silhouette Score
- 記錄計算時間
- **分析**: 哪種核函數最適合？非線性核是否有明顯優勢？

### **實驗3: t-SNE 參數調整影響**

比較不同參數組合的效果：
- **perplexity**: 5, 30, 50, 100
- **learning_rate**: 10, 200, 1000
- **n_iter**: 250, 1000, 5000

**要求**:
- 固定其他參數，逐個測試每個參數的影響
- 繪製不同參數下的2D散點圖
- 記錄KL散度（如果可用）
- 觀察收斂速度和結果穩定性
- **分析**: 參數選擇如何影響結果？給出最佳參數組合建議

### **實驗4: 降維 + 監督式學習整合**

探索降維作為特徵工程對監督式學習的影響：
- **情境**: 假設操作模式標籤已知，要訓練分類器
- 比較以下情況的分類準確率：
  - 使用原始20個特徵
  - 使用PCA降維後的2個特徵
  - 使用PCA降維後的5個特徵
  - 使用PCA降維後的10個特徵
  - 使用Kernel PCA降維後的5個特徵

**要求**:
- 使用相同的分類器（如Logistic Regression或Random Forest）
- 進行交叉驗證
- 記錄準確率、訓練時間
- 繪製ROC曲線或混淆矩陣
- **分析**: 降維如何影響監督式學習？何時應該使用降維？

### 💭 思考題

1. 為什麼在進行降維之前需要對數據進行標準化？
2. PCA的主成分有什麼物理意義？如何解釋主成分？
3. 線性降維（PCA）和非線性降維（Kernel PCA, t-SNE, UMAP）各適合什麼場景？
4. t-SNE為什麼不適合用於預測新數據？UMAP是否有同樣的限制？
5. 如何選擇合適的降維維度？保留多少個主成分才足夠？
6. 在化工製程監控中，降維技術有哪些實際應用？
7. 降維會損失信息，為什麼我們還要使用降維？
8. Silhouette Score如何評估降維後的數據品質？
9. 為什麼t-SNE的perplexity參數會影響結果？如何選擇合適的值？
10. 在實際工業應用中，如何平衡降維的計算成本和效果？

---
# 想對老師說的話